Activity 2.6 -- Working with COVID 19 and World Bank Data
=========================================================

In this activity, you will explore relationships between various World
Bank indicators for countries and their corresponding COVID death rates.
First you need to download data on COVID-19 (see links and instructions
below) and the selected indicators from the Open World Bank data
available at <https://data.worldbank.org>.

**COVID data set source:** <https://coviddata.github.io/coviddata/#csvs>

**Tasks.** Use pandas and dfply to perform each of the following.

1.  Download the raw **time\_series\_covid19\_confirmed\_global.csv**
    dataset.

2.  Inspect the data and discuss the need to reshape. 

In [2]:
# Code for loading and inspecting the CSV file
import pandas as pd
from dfply import *

In [3]:
covid = pd.read_csv("/home/wil/activity_2_5_COVID_and_World_Bank_Data/data/time_series_covid19_confirmed_global.txt")

In [4]:
covid.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/3/22,9/4/22,9/5/22,9/6/22,9/7/22,9/8/22,9/9/22,9/10/22,9/11/22,9/12/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,193912,194163,194355,194614,195012,195298,195471,195631,195925,196182
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,330062,330193,330221,330283,330516,330687,330842,330948,331036,331053
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,270426,270443,270461,270476,270489,270507,270522,270532,270539,270551
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,46027,46027,46027,46027,46113,46113,46113,46113,46113,46113
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,102636,102636,102636,102636,102636,102636,103131,103131,103131,103131


> **Your discussion:**

there are 900 plus dates on this 

In [5]:
coviddate = (covid
        >> gather ("date", "covid_deaths", columns_from('1/22/20'))
        >> mutate (date = X.date.astype('datetime64'))
        >> mutate (year = X.date.dt.year,
                   month = X.date.dt.month))
coviddate.head()

,Province/State,Country/Region,Lat,Long,date,covid_deaths,year,month
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,2020,1
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,2020,1
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,2020,1
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,2020,1
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,2020,1


3.  Write a single pipe that reshapes the data, sets the dtype of the date column, and extracts various date parts.
    1. To change the `dtype` of the date column, `date = X.date.astype('datetime64')`
    2. To extract the year and month, use the `X.date.dt.year` and `X.date.dt.month` attributes. This will need to happen in a separate `mutate` 

### World Bank Links Development Indicators

<https://databank.worldbank.org/source/world-development-indicators>

#### Constructing a data set.

First you need to construct a data set as follows

1.  Expand the Country tab and select all.

<img src="./img/media/image1.png" width="300">

2.  Click on the Series tab, search for *Health* and select the
    following indicators. **Feel free to add additional indicators!**

<img src="img/media/image2.png" width="300">

3.  Click on the Time tab and select 2020 and 2021.

4.  Click apply changes in the floating dialog.

<img src="img/media/image3.png" width="300">

5.  Select CSV from the Download Options button and save the data folder

<img src="img/media/image4.png" width="100">

#### Tasks

Use pandas and dfply to perform each of the following.

1.  Inspect the World Bank data and discuss the need to reshape. 

**Hints:** 

* You should apply `fix_names` from `more_dfply` to clean up the column names.
* This table needs to be reshaped twice




In [6]:
from more_dfply import fix_names

In [7]:
# Code for loading and inspecting the CSV
world_dev = pd.read_csv("/home/wil/activity_2_5_COVID_and_World_Bank_Data/data/world_development_data.csv") >> fix_names
world_dev.head()

,Country_Name,Country_Code,Series_Name,Series_Code,_2018_YR2018,_2019_YR2019
0,Afghanistan,AFG,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,14.12674332,13.24220181
1,Afghanistan,AFG,Domestic general government health expenditure...,SH.XPD.GHED.GD.ZS,0.54922014,1.08443093
2,Afghanistan,AFG,Domestic general government health expenditure...,SH.XPD.GHED.PC.CD,2.72140895,5.38899046
3,Afghanistan,AFG,Domestic private health expenditure (% of curr...,SH.XPD.PVTD.CH.ZS,76.319664,79.39915466
4,Afghanistan,AFG,Domestic private health expenditure per capita...,SH.XPD.PVTD.PC.CD,53.42270666,52.24943995


In [8]:
world_dev.shape

(1335, 6)

> **Your discussion:**

2.  Write a single pipe that reshapes the data, sets the dtype of the date column, and extracts various date parts.
    1. You can use the `replace` method to clean up the year column.  See lecture 3.1 for details.

In [39]:
year_reshape = (world_dev
              >> select(~X.Series_Code)
              >> gather('year', 'value', columns_from('_2018_YR2018'), add_id = True)
              >> filter_by(X._ID < 1330)
              >> select(~X._ID)
              >> spread(X.Series_Name, X.value)
              >> mutate(year = X.year.str.replace('_2018_YR2018', '2018'))
              >> mutate(year = X.year.str.replace('_2019_YR2019', '2019'))
              >> mutate(year = X.year.astype('datetime64'))
              >> mutate(year = X.year.dt.year)
              )
year_reshape

,Country_Name,Country_Code,year,Current health expenditure (% of GDP),Domestic general government health expenditure (% of GDP),Domestic general government health expenditure per capita (current US$),Domestic private health expenditure (% of current health expenditure),Domestic private health expenditure per capita (current US$)
0,Afghanistan,AFG,2018,14.12674332,0.54922014,2.72140895,76.319664,53.42270666
1,Afghanistan,AFG,2019,13.24220181,1.08443093,5.38899046,79.39915466,52.24943995
2,Africa Eastern and Southern,AFE,2018,6.28682941150035,3.01138620120092,45.00322636148,40.2746316114941,38.7747764079056
3,Africa Eastern and Southern,AFE,2019,6.26638563294957,3.04368546715315,43.8300388099369,39.7848380711114,36.5804196112588
4,Africa Western and Central,AFW,2018,3.39316031669726,0.74466519924604,12.909945972337,67.0338193214843,39.7703675274289
...,...,...,...,...,...,...,...,...
527,"Yemen, Rep.",YEM,2019,..,..,..,..,..
528,Zambia,ZMB,2018,5.03271484,1.96286535,29.76423856,16.62527084,12.68747921
529,Zambia,ZMB,2019,5.31220293,2.12920475,27.78617575,16.21423721,11.24041905
530,Zimbabwe,ZWE,2018,8.68006228,2.78293462,45.32473722,42.76859662,60.46166901


### Investigate joining on country

Before we can proceed, we need to make sure that the columns used to join the data--namely the country--actually match.  Do this by

1. For each table, select just the country columns and make sure the column names match.
2. Perform a full outer join and filter on rows that didn't match (i.e. with a missing value in either column).
3. Determine any transformations needed to make the entries match.
4. Transform each of the original table as need (column names and/or problematic entries.

In [41]:
# Your code here

dates = (coviddate
         >> rename (Country_Name = 'Country/Region')
         >> select (X.Country_Name)
         >> mutate (Country2 = X.Country_Name))

In [45]:
tbl_join = (world_dev
            >> select (X.Country_Name)
            >> mutate (Country3 = X.Country_Name)
            >> outer_join(dates, by='Country_Name')
            >> filter_by(X.Country2.isna() | X.Country3.isna()))
tbl_join.Country_Name.unique()

fix_country = (tbl_join
               >> mutate(Country_Name = X.Country_Name.str.replace('Bahamas, The', 'Bahamas'))
               >> mutate(Country_Name = X.Country_Name.str.replace('Brunei Darussalam', 'Brunei'))
               >> mutate(Country_Name = X.Country_Name.str.replace('Congo, Dem. Rep', 'Republic_Congo'))
               >> mutate(Country_Name = X.Country_Name.str.replace('')


array(['American Samoa', 'Aruba', 'Bahamas, The', 'Bermuda',
       'British Virgin Islands', 'Brunei Darussalam', 'Cayman Islands',
       'Channel Islands', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Curacao',
       'Czech Republic', 'Egypt, Arab Rep.', 'Faroe Islands',
       'French Polynesia', 'Gambia, The', 'Gibraltar', 'Greenland',
       'Guam', 'Hong Kong SAR, China', 'Iran, Islamic Rep.',
       'Isle of Man', "Korea, Dem. People's Rep.", 'Korea, Rep.',
       'Kyrgyz Republic', 'Lao PDR', 'Macao SAR, China',
       'Micronesia, Fed. Sts.', 'Myanmar', 'Nauru', 'New Caledonia',
       'Northern Mariana Islands', 'Puerto Rico', 'Russian Federation',
       'Sint Maarten (Dutch part)', 'Slovak Republic',
       'St. Kitts and Nevis', 'St. Lucia', 'St. Martin (French part)',
       'St. Vincent and the Grenadines', 'Syrian Arab Republic',
       'Turkiye', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu',
       'United States', 'Venezuela, RB', 'Virgin Islands (U.S.)',
       'Yem

In [46]:
fix_country = (tbl_join
               >> mutate(Country_Name = X.Country_Name.str.replace('Bahamas, The', 'Bahamas'))
               >> mutate(Country_Name = X.Country_Name.str.replace('')))

## Join and visualize 

Finally, you should use pandas and dfply to join these two data sets together, then create some interesting visualization using seaborn.

In [ ]:
# Your code here



### Deliverables
To complete this part of the activity, you need to submit the following.

1.  A link to this notebook including all discussion and code requests
    above.

2.  A csv file containing your final dataset. **Hint.** You can use the
    to\_csv method on the final data frame.

In [ ]:
# Code for writing the data here